In [16]:
import pandas as pd
#df = pd.read_json("Electronics_5.json",lines=True)
df = pd.read_json("Musical_Instruments_5.json",lines=True)
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1384719342,"[0, 0]",5,"Not much to write about here, but it does exac...","02 28, 2014",A2IBPI20UZIR0U,"cassandra tu ""Yeah, well, that's just like, u...",good,1393545600
1,1384719342,"[13, 14]",5,The product does exactly as it should and is q...,"03 16, 2013",A14VAT5EAX3D9S,Jake,Jake,1363392000
2,1384719342,"[1, 1]",5,The primary job of this device is to block the...,"08 28, 2013",A195EZSQDW3E21,"Rick Bennette ""Rick Bennette""",It Does The Job Well,1377648000
3,1384719342,"[0, 0]",5,Nice windscreen protects my MXL mic and preven...,"02 14, 2014",A2C00NNG1ZQQG2,"RustyBill ""Sunday Rocker""",GOOD WINDSCREEN FOR THE MONEY,1392336000
4,1384719342,"[0, 0]",5,This pop filter is great. It looks and perform...,"02 21, 2014",A94QU4C90B1AX,SEAN MASLANKA,No more pops when I record my vocals.,1392940800


In [17]:
#pre-processing

#dropping useless columns for fitting
df = df.drop(columns=['helpful', 'reviewText', 'reviewTime', 'reviewerName','summary','unixReviewTime'])

#rearrange the order of columns
df = df[['reviewerID','asin','overall']]

df.head()

,reviewerID,asin,overall
0,A2IBPI20UZIR0U,1384719342,5
1,A14VAT5EAX3D9S,1384719342,5
2,A195EZSQDW3E21,1384719342,5
3,A2C00NNG1ZQQG2,1384719342,5
4,A94QU4C90B1AX,1384719342,5


In [73]:
#statics about the data
print ("Total review count:" + str(df.shape[0]))
print ("Mean of reviews:" + str(df.overall.mean()))
unique_reviewer = df['reviewerID'].nunique()
print("Number of reviewers:"+str(unique_reviewer))
unique_asin = df['asin'].nunique()
print("Number of items:"+str(unique_asin)) 

Total review count:10261
Mean of reviews:4.488743787155248
Number of reviewers:1429
Number of items:900


In [5]:
from surprise import Dataset
from surprise import Reader

#load data to surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df, reader)

data.raw_ratings[:5]

[('A2IBPI20UZIR0U', '1384719342', 5.0, None),
 ('A14VAT5EAX3D9S', '1384719342', 5.0, None),
 ('A195EZSQDW3E21', '1384719342', 5.0, None),
 ('A2C00NNG1ZQQG2', '1384719342', 5.0, None),
 ('A94QU4C90B1AX', '1384719342', 5.0, None)]

In [6]:
from surprise import KNNWithMeans
from surprise import BaselineOnly
from surprise import SVD
from surprise import NMF
from surprise.model_selection import cross_validate
                                                                                                   
#cross-validation evaluation for different alg

print("KNN with ratings of each user:")
algo = KNNWithMeans()                  
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print("="*80)

print("Baseline only method:")
algo = BaselineOnly()                  
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print("="*80)

print("Matrix Factorization SVD:")
algo = SVD()                  
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print("="*80)

print("Matrix Factorization NMF:")
algo = NMF()                  
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print("="*80)

KNN with ratings of each user:
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0015  1.0139  1.0063  1.0054  0.9976  1.0049  0.0054  
MAE (testset)     0.6567  0.6933  0.6724  0.6670  0.6503  0.6679  0.0148  
Fit time          0.09    0.09    0.09    0.09    0.08    0.09    0.00    
Test time         0.07    0.08    0.08    0.08    0.07    0.07    0.00    
Baseline only method:
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8823  0.8635  0.8619  0.8435  0.8898  0.8682  0.0164  
MAE (testset)     0.6552  0.6453  0.6417  0.6308  0.6529  0.6452  0.0087  
Fit time          0.01    0.01    0.01    0.01    0.01    0.01    0.00    
Test time         0.02    0.02    0.02    0.02    0.02    0.02    0.00    
Matrix Factorization SVD:
Evaluating RMSE, MAE of algorithm SVD on 5 sp

In [7]:
#From the comparsion above, we use SVD algorithm

from collections import defaultdict

#function from Surprise documentation
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

#train data
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=3)

for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

A2IBPI20UZIR0U ['B0001FTVD6', 'B0002BACB4', 'B0002CZST4']
A14VAT5EAX3D9S ['B0002BACB4', 'B0002CZVB4', 'B0002CZVHI']
A195EZSQDW3E21 ['B000068O3D', 'B000EEHN9S', 'B000U0DU34']
A2C00NNG1ZQQG2 ['B0015RIN6U', 'B0002E5518', 'B001W99HE8']
A94QU4C90B1AX ['B001W99HE8', 'B002MWKOAA', 'B0002GIRP2']
A2A039TZMZHH9Y ['B0002E2XCW', 'B000GUR8V8', 'B0015RIN6U']
A1UPZM995ZAH90 ['B0002CZUUG', 'B0002CZV46', 'B0002CZV78']
AJNFQI3YR6XJ5 ['B000A6ASSS', 'B001J1JRN0', 'B000VJZOLK']
A3M1PLEYNDEYO8 ['1384719342', 'B000068O3D', 'B0001FTVD6']
AMNTZU1YQN1TH ['B0002IHGZC', 'B0033P1O6S', 'B0002GZBLU']
A2NYK9KWFMJV4Y ['1384719342', 'B000068NSX', 'B000068O1N']
A35QFQI0M46LWO ['B0002DV7ZM', 'B002433RXI', 'B0033P1O6S']
A2NIT6BKW11XJQ ['B0002E2XCW', 'B000068O3X', 'B000RKL8R2']
A1C0O09LOLVI39 ['B0015RIN6U', 'B000960KNW', 'B0002DURNK']
A17SLR18TUMULM ['B000LAT0AK', 'B00923G9Q0', 'B0002E2XCW']
A2PD27UKAD3Q00 ['B0033P1O6S', 'B0042EZH6W', 'B000NJ2TIE']
AKSFZ4G1AXYFC ['B000RY68PA', 'B005MR6IHK', 'B009MIBIWK']
A67OJZLHBBUQ9 ['13

A2SNKWR5D203ZX ['B0000AQRST', 'B0001FTVD6', 'B0002CZVHI']
A19AXCVXAM7NM2 ['B0002GWFEQ', 'B0027842S4', 'B0042EZH6W']
A1LCG14GZJI5LO ['B000RYPN38', 'B000RKL8R2', 'B0002D0CEO']
A1FO10Z93GKN33 ['B0002GLCRC', 'B0015RIN6U', 'B005M0TKL8']
A3D1A0YHTT0E9G ['B000L6GD04', 'B0037M62KA', 'B0002E2XCW']
A2Y4PRG2HHOL3G ['B001W99HE8', 'B0015RIN6U', 'B000A6ASSS']
A1ZQVN0F0FURHH ['B000068O3X', 'B0002E2XCW', 'B0002E52RU']
ACWH4WJMC5UFN ['B0015RIN6U', 'B0002D01K4', 'B0033P1O6S']
ANZGC4YCEQC6W ['B0002E2XCW', 'B001W99HE8', 'B000L6GD04']
A1F5HMIDLSR3EF ['B001W99HE8', 'B0037M62JQ', 'B0042EZH6W']
A3EH9SDPD1FGEA ['B0015RIN6U', 'B001FQ74FW', 'B0042EZH6W']
A30HTJ530SMRB7 ['B001W99HE8', 'B0015RIN6U', 'B00C5B20QE']
A2JZY5POMJADL1 ['B000068O3X', 'B0001FTVD6', 'B0002D0MFI']
A3G6KKDMMFQQ5K ['B000LQLDM2', 'B0073XCXHA', 'B0002D0KOG']
A3HKX5UIDJ403N ['B0002D0CEO', 'B0002D0KOG', 'B0002E1NQ4']
ASEBPAY8IBNI2 ['B0015RIN6U', 'B002ZH9JQC', 'B00H02C9TG']
AA4BAIDEDJI3 ['B0027842S4', 'B0007GGUGA', 'B0002BACB4']
A2H629SI5VJGRM ['B0

In [2]:
import sys
import urllib.request
import copy
from bs4 import BeautifulSoup

#get name of product from asin
def get_name(asin):
    url = "https://www.amazon.com/dp/"+asin
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    req = urllib.request.Request(url,None, headers)
    try: 
        soup = BeautifulSoup(urllib.request.urlopen(req).read(), 'html.parser')
        span = soup.find("span", id="productTitle")
        return span.text.strip()
    except:
        return get_name(asin)


In [3]:
import mysql.connector
from mysql.connector import errorcode
#Inserting products into local database because of MSU mysql connection policy
uname = "root"
pwd = ""
hname = "localhost"

grp = df.groupby(['asin'])
i = 0
for n,g in grp:
    asin = n
    avg = g['overall'].mean()
    
    try:
        cnx2 = mysql.connector.connect(user=uname, password=None, host=hname, database="duzengze")
        cursor2 = cnx2.cursor()
        query = "SELECT * FROM 482_product WHERE id=\'" +asin +"\'"
#         print(i)
        cursor2.execute(query)
        cursor2.fetchall()
        i +=1
#         print(cursor2.rowcount)
        if cursor2.rowcount != 0:
            cursor2.close()
            continue
        cursor2.close()
    except mysql.connector.Error as e:
        print(e.msg )       # error message
    else:
        cnx2.commit()
        cnx2.close()
#     print(asin)
    name = get_name(asin)
#     print(name)
    name = name.replace("'", "\\'")
    name = name.replace('"', '\\"')
    query = "INSERT INTO 482_product VALUES(\'"+ asin +"\',\'"+ name +"\',\'"+ str(avg) +"\')"
    try:
        cnx = mysql.connector.connect(user=uname, password=None, host=hname, database="duzengze")
        cursor = cnx.cursor()
        cursor.execute(query)
        cursor.close()
    except mysql.connector.Error as e:
        print(e.msg )       # error message
    else:
        cnx.commit()
        cnx.close()
        
# Then export the table from local database to msu database

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

Fender Clip-On Tuner FT-004 for Guitar, Ukulele, Bass, Violin, Mandolin, and Banjo
name
800
B005NACC6M
TASCAM DR-40 4-Track Portable Digital Recorder
name
801
B005OZE9SA
Focusrite Scarlett 2i2 (1st GENERATION) USB Recording Audio Interface
name
802
B005PGGU9O
Simtyso®  Predmium Quick Change Folk Acoustic Electric Guitar Banjo Trigger Capo Key Clamp
name
803
B005PNXT6E
Fender Performance Series Instrument Cables (1/4 Straight-to-Straight) for electric guitar, bass guitar, electric mandolin, pro audio
name
804
B005SUS1JO
Fender Steel Pick (4 Pack)
name
805
B005T800V2
Perfektion Pack of six different colors, 1 foot Guitar - instrument Patch Cables (Right Angle) - Can be used for FX (effects) pedals, Instruments, and More
name
806
B005TO9EPY
Bluecell 5 Pack Blue/Green/Yellow/Hot Pink/Orange Handheld Stage Microphone Windscreen Foam Cover
name
807
B005VLR6XC
Leegoal White Single-handed Guitar Capo Quick Change
name
808
B005VLWHP4
HIGHROCK Capo Single Handed Guitar Capo Quick Change for Guit

Planet Waves NS Artist Guitar Capo, Black
name
895
B00H02C9TG
Apogee JAM 96k Guitar Interface Input for iPad, iPhone and Mac
name
896
B00HFRXACG
D'Addario EFX165 FlexSteels Bass Guitar Strings, Custom Light, 45-105, Long Scale
name
897
B00IZCSW3M
D'Addario NYXL0942 Nickel Plated Electric Guitar Strings, Extra Light
name
898
B00J4TBMVO
Elixir Strings 80/20 Bronze Acoustic Guitar Strings w NANOWEB Coating, HD Light (.013-.053)
name
899
B00JBIVXGC
Elixir Strings Phosphor Bronze Acoustic Guitar Strings w NANOWEB Coating, HD Light (.013-.053)
name


In [66]:
#Get our custom reviews from database
request = urllib.request.urlopen('http://webdev.cse.msu.edu/~duzengze/recommendation/dsb.php')
reviews = request.read().decode("utf-8") 
review_list = reviews.split("|")
del review_list[-1]
new_list = []
for review in review_list:
    new_list.append(review.split(','))
new_list

new_df = pd.DataFrame( new_list, columns = ['reviewerID','asin','overall'])
new_pd = pd.concat([new_df,df])
new_pd.head()

from surprise import Dataset
from surprise import Reader
from surprise import SVD

#load data to surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(new_pd, reader)

data.raw_ratings[:10]



[('1', '1384719342', 5.0, None),
 ('1', 'B000068NSX', 5.0, None),
 ('1', 'B000068O1N', 2.0, None),
 ('1', 'B00009W40G', 1.0, None),
 ('1', 'B0002CZSJY', 5.0, None),
 ('1', 'B0002D0B4K', 2.0, None),
 ('1', 'B0002D0LKY', 2.0, None),
 ('1', 'B0002GML4U', 1.0, None),
 ('1', 'B0002Z2D08', 1.0, None),
 ('A2IBPI20UZIR0U', '1384719342', 5.0, None)]

In [41]:
#function to save data back to our database

def save_recommend(uid,r):
    #retry 100 times if exception happens
    for i in range(100):
        try:
            urllib.request.urlopen('http://webdev.cse.msu.edu/~duzengze/recommendation/nmsl.php?uid='+uid+'&first='+r[0]+'&second='+r[1]+'&third='+r[2])
            break
        except:
            continue
            
    


In [72]:
#Re-fitting the dataset to find the recommandation

from collections import defaultdict
from surprise import accuracy

#function from Surprise documentation
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

#train data
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

testset = trainset.build_anti_testset()
predictions = algo.test(testset)
acc = accuracy.rmse(predictions, verbose=True)

ids = df.reviewerID.tolist()

top_n = get_top_n(predictions, n=3)

for uid, user_ratings in top_n.items():
    r = [iid for (iid, _) in user_ratings]
    if uid in ids:
        continue
    save_recommend(uid,r)
    
    

RMSE: 0.2911
